<a href="https://colab.research.google.com/github/xman227/SWbootcamp/blob/main/%EA%B9%80%EC%A7%80%ED%9B%88/drug_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 경구 약제 이미지 분류

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, MaxPool2D
from keras.callbacks import ModelCheckpoint, EarlyStopping

from PIL import Image
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt

import os
import glob

In [2]:
tf.version

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.8/dist-packages/tensorflow/_api/v2/version/__init__.py'>

10개 종류 약제를 분류하자

In [3]:
!unzip -q -o /content/drive/MyDrive/drug_data.zip

In [4]:
#변수 설정
img_width = 64
img_height = 64
img_dir = '/content/image'
label_dir = "/content/label"
drive = '/content/drive/MyDrive'

In [5]:
#폴더 명
imageDirNames = list(os.scandir(img_dir))

#label 설정([K-039306, K-039320, ...])
drug_label = [drug.name for drug in imageDirNames]
print(drug_label)

['K-039393', 'K-039337', 'K-039504', 'K-039478', 'K-039306', 'K-039320', 'K-039325', 'K-039338', 'K-039362', 'K-039346']


In [ ]:
#label 데이터를 불러오는 함수
#필요한가?

def getJson(drugPath):
    with open(drugPath, 'r') as j:
        contents = json.loads(j.read())

    #print(contents)
    return contents

getJson(label_dir + "/K-039306_json/K-039306_0_0_0_0_75_000_200.json")['images']


[{'file_name': 'K-039306_0_0_0_0_75_000_200.png',
  'width': 976,
  'height': 1280,
  'imgfile': 'K-039306_0_0_0_0_75_000_200.png',
  'drug_N': 'K-039306',
  'drug_S': '정상알약',
  'back_color': '검은색 배경',
  'drug_dir': '앞면',
  'light_color': '전구색',
  'camera_la': 75,
  'camera_lo': 0,
  'size': 200,
  'dl_idx': '39305',
  'dl_mapping_code': 'K-039306',
  'dl_name': '탐루신디서방정',
  'dl_name_en': 'Tamlusindi SR Tab.',
  'img_key': 'http://connectdi.com/design/img/drug/147618960468200180.jpg',
  'dl_material': '탐스로신염산염',
  'dl_material_en': 'Tamsulosin Hydrochloride',
  'dl_custom_shape': '서방형정제',
  'dl_company': '(주)휴온스',
  'dl_company_en': 'Huons',
  'di_company_mf': '(주)동구바이오제약',
  'di_company_mf_en': 'Dongkoo Bio&Pharma',
  'item_seq': 201603317,
  'di_item_permit_date': '20160609',
  'di_class_no': '[02590]기타의 비뇨생식기관 및 항문용약',
  'di_etc_otc_code': '전문의약품',
  'di_edi_code': '670607510',
  'chart': '흰색의 원형 필름코팅정제',
  'drug_shape': '원형',
  'thick': 3.78,
  'leng_long': 9.2,
  'leng_short': 9.2

In [18]:
#이미지 전처리

X_data = []
Y_data = []
a = ""
Image.LOAD_TRUNCATED_IMAGES = True
print(drug_label)
try:
    for drugName in drug_label:
        #print(*glob.glob(f'{img_dir}/{drugName}/*.png'), sep = '\n')

        print(drugName)
        for drugImageName in glob.glob(f'{img_dir}/{drugName}/*.png'):
            a = drugImageName
            #print(drugImageName)
            img = Image.open(drugImageName)
            img = img.convert("RGB")
            img = img.resize((img_width, img_height))
            X_data.append(np.array(img))
            Y_data.append(drug_label.index(drugName))
            #display(img)
except:
    #잘못된 이미지 검출
    print(a)






['K-039393', 'K-039337', 'K-039504', 'K-039478', 'K-039306', 'K-039320', 'K-039325', 'K-039338', 'K-039362', 'K-039346']
K-039393
K-039337
K-039504
K-039478
K-039306
K-039320
K-039325
K-039338
K-039362
K-039346


In [19]:
print(len(X_data))

# 이미지 배열 저장
np.save(drive + '/x_drug.npy', tf.keras.utils.normalize(X_data))
np.save(drive + '/y_drug.npy', Y_data)

2268


In [20]:
#전처리 후 저장된 데이터 읽어오기
X_data = np.load(drive+'/x_drug.npy')
Y_data = np.load(drive+'/y_drug.npy')


#훈련 데이터 분할
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size = 0.2, random_state=100)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.2, random_state=100)
print('훈련 샘플 : %d개' %len(X_train))
print('검증 샘플 : %d개' %len(X_val))
print('테스트 샘플 : %d개' %len(X_test))

print(X_train.shape)
print(Y_train.shape)

훈련 샘플 : 1451개
검증 샘플 : 363개
테스트 샘플 : 454개
(1451, 64, 64, 3)
(1451,)


In [21]:
Y_train

array([2, 2, 7, ..., 0, 9, 2])

In [23]:
#모델(저번에 쓰던 모델 그대로)

model = Sequential()

model.add(Conv2D(32,(3,3), input_shape=(img_width,img_height,3), activation='relu'))
model.add(MaxPooling2D((3,3)))
#model.add(Dropout(0.25))
model.add(Conv2D(64, (2,2), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D((2,2)))
model.add(MaxPool2D(2))
model.add(Conv2D(128, kernel_size=3, activation='relu', padding='same'))
model.add(Flatten())
model.add(Dense(len(drug_label), activation = 'sigmoid'))
model.summary()

model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer='adam',
              metrics = ['accuracy'])

MODEL_DIR = "./model/"

if not os.path.exists(MODEL_DIR):
  os.mkdir(MODEL_DIR)

modelpath = './model/{epoch:02d}-{val_loss:.4f}.hdf5'
checkpointer = ModelCheckpoint(filepath=modelpath, monitor = 'val_loss', verbose=1, save_best_only=True)

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)


#학습
history = model.fit(X_train, Y_train,
                    validation_data = (X_val, Y_val),
                    epochs=50,
                    verbose=1,
                    batch_size = 20,
                    callbacks=[early_stopping_callback, checkpointer])

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 20, 20, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 20, 20, 64)        8256      
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 10, 10, 64)       0         
 g2D)                                                            
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 5, 5, 64)         0         
 g2D)                                                            
                                                      

In [24]:
#모델 검증(저번에 쓰던 모델 그대로)

val_eval = model.evaluate(X_val, Y_val)
print("\n Val Loss: %.4f" % (val_eval[0]))
print("\n Val Accuracy: %.4f\n" % (val_eval[1]))
test_eval = model.evaluate(X_test, Y_test)
print("\n Test Loss: %.4f" % (test_eval[0]))
print("\n Test Accuracy: %.4f" % (test_eval[1]))

12/12 [==============================] - 0s 4ms/step - loss: 0.1829 - accuracy: 0.9229

 Val Loss: 0.1829

 Val Accuracy: 0.9229

15/15 [==============================] - 0s 8ms/step - loss: 0.1672 - accuracy: 0.9383

 Test Loss: 0.1672

 Test Accuracy: 0.9383


In [ ]:
#파인튜닝

